In [19]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch, helpers


In [20]:
vnexpress = {
    "life": ["https://vnexpress.net/doi-song/cooking",
           "https://vnexpress.net/doi-song/to-am",
           "https://vnexpress.net/doi-song/bai-hoc-song",
           "https://vnexpress.net/doi-song/nha",
           "https://vnexpress.net/doi-song/tieu-dung"],
    "news": ["https://vnexpress.net/thoi-su/chinh-tri",
           "https://vnexpress.net/thoi-su/dan-sinh",
           "https://vnexpress.net/thoi-su/giao-thong",
           "https://vnexpress.net/thoi-su/mekong",
           "https://vnexpress.net/thoi-su/quy-hy-vong"],
    "world": ["https://vnexpress.net/the-gioi/quan-su",
            "https://vnexpress.net/the-gioi/tu-lieu",
            "https://vnexpress.net/the-gioi/phan-tich",
            "https://vnexpress.net/the-gioi/nguoi-viet-5-chau",
            "https://vnexpress.net/the-gioi/cuoc-song-do-day"]
}


In [21]:
vnexpressLinks = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in vnexpress.items():
    for url in value:
        response = requests.get(url)
        tree = lxml.html.fromstring(response.content)
        title_elem = tree.cssselect('.title-news a')
        for elem in title_elem:
            vnexpressLinks[key].append(elem.get('href'))

print(vnexpressLinks)

{'life': ['https://vnexpress.net/3-sai-lam-nuoi-con-cua-cha-me-ho-4447701.html', 'https://vnexpress.net/5-tai-hai-khi-kiem-soat-con-qua-muc-4447362.html', 'https://vnexpress.net/chi-bon-tien-cho-con-hoc-o-khach-san-hang-sang-4446835.html', 'https://vnexpress.net/vo-chong-chung-tai-khoan-se-hanh-phuc-hon-4446734.html', 'https://vnexpress.net/ket-co-hau-cua-cau-be-hai-lan-bi-co-giao-tra-ve-4445537.html', 'https://vnexpress.net/nhat-ban-tra-70-chi-phi-ivf-de-khuyen-khich-sinh-de-4446547.html', 'https://vnexpress.net/thiet-thoi-kep-cua-tre-tu-ky-trong-dai-dich-4445922.html', 'https://vnexpress.net/cha-me-nen-lam-gi-giup-tre-doc-lap-tu-tin-4445846.html', 'https://vnexpress.net/the-gioi-cua-nhung-gia-dinh-co-con-tu-ky-4445541.html', 'https://vnexpress.net/ban-cong-ty-dua-cha-nuoi-di-du-lich-khap-trung-quoc-4445800.html', 'https://vnexpress.net/nhung-ong-chong-bi-bao-hanh-4445239.html', 'https://vnexpress.net/chong-ham-nhau-4444836.html', 'https://vnexpress.net/hoi-han-khi-voi-ket-hon-vi-so-e

In [22]:
vnexpressContent = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in vnexpressLinks.items():
    for url in value:
        if(url.startswith('https://vnexpress.net/')):
            id = url.split('-')[-1].replace('.html', '')
            downloaded = trafilatura.fetch_url(url)
            tree = lxml.html.fromstring(str(downloaded))
            date_elem = len(tree.cssselect(".date")) and tree.cssselect(".date")[0].text or ""
            content = trafilatura.extract(downloaded)
            content = content.replace('\n', ' ')
            item = {"domain": "https://vnexpress.net", "URL": url,"content": content, "date": date_elem,"id": id}
            vnexpressContent[key].append(item)

print(len(vnexpressContent['life'])+len(vnexpressContent['news'])+len(vnexpressContent['world']))

410


In [23]:
client = Elasticsearch("http://localhost:9200")

client.info()


{'name': 'HoangPhiLong-20182651',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'jF2zfdDdQqueQik2u6h9Ug',
 'version': {'number': '7.17.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': 'de7261de50d90919ae53b0eff9413fd7e5307301',
  'build_date': '2022-03-28T15:12:21.446567561Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [27]:
def gendata():
    for key,value in vnexpressContent.items():
         for item in value:
            temp_item = {k: item[k] for k in set(list(item.keys())) - set(['id'])}
            temp_item['category'] = key
            client.index(index="journal", id=item['id'], document=temp_item)

gendata()

/home/hplong/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
